In [ ]:
nrows = None
iterations = None

run = 21

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import numpy as np 
import pandas as pd
from sklearn.model_selection import StratifiedKFold, KFold, TimeSeriesSplit
from sklearn.metrics import accuracy_score
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split as train_eval_split
from lightgbm import LGBMClassifier
import pdb
from sklearn.preprocessing import StandardScaler, LabelEncoder
from math import factorial
import re
from catboost import CatBoostRegressor
from collections import defaultdict
from lightgbm import LGBMRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline

In [ ]:
def train_eval_kfolds(X, y, n_splits=5, random_state=None):
    assert hasattr(X, 'iloc') # needs to be pandas data
    assert hasattr(y, 'iloc')
    skf = TimeSeriesSplit(n_splits=n_splits) #, shuffle=True, random_state=random_state)
    for train_index, eval_index in skf.split(X, y):
        X_train, X_eval = X.iloc[train_index], X.iloc[eval_index]
        y_train, y_eval = y.iloc[train_index], y.iloc[eval_index]
        yield X_train, X_eval, y_train, y_eval

In [ ]:
def isnotebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

In [ ]:
train_df = pd.read_csv(
    '../input/tabular-playground-series-mar-2022/train.csv', index_col='row_id', nrows=nrows)
train_df.head()

In [ ]:
pd.to_datetime(train_df['time'])

In [ ]:
train_df['congestion']

In [ ]:
if isnotebook():
    train_df['congestion'].hist(bins=100)

In [ ]:
target_mean = train_df['congestion'].mean()
target_mean

In [ ]:
(train_df['congestion'] - target_mean).abs().mean() # s/b 13.87 for starters.

In [ ]:
train_df['x'].unique()

In [ ]:
train_df['y'].unique()

In [ ]:
train_df[['x', 'y']].drop_duplicates().shape # only 12 locations.  Maybe do 12 models

In [ ]:
locations = []
for x_loc in train_df['x'].unique():
    for y_loc in train_df['y'].unique():
        locations.append((x_loc, y_loc))
locations

In [ ]:
features = [c for c in train_df.columns if c not in ['congestion']]

X = train_df[features]
y = train_df['congestion']
features

In [ ]:
class HandleDates(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        X2 = X.copy()
        date_series = pd.to_datetime(X['time'])

        date_accessor = date_series.dt
        for attr in ['date', 
                     'day', 'day_of_week', 'day_of_year', 
                     'days_in_month', 'freq', 'hour', 
                     'is_leap_year', 'is_month_end', 'is_month_start', 'is_quarter_end', 
                     'is_quarter_start', 'is_year_end', 'is_year_start', 
                     'microsecond', 'minute', 'month', 'nanosecond', 
                      'quarter', 'second', 'time', 
                     'weekday', 'year'
                    ]:
            X2[attr] = getattr(date_accessor, attr)
        X2['week'] = date_accessor.isocalendar().week.astype(int)
        X2['original_time'] = X['time']
        X2['time'] = X2['time'].astype(str)
        
        # add location as one column
        # this makes the eval better and kaggle worse. Commenting out for now.
#         X2['location'] = X2[["x","y"]].apply(tuple, axis=1)
        return X2

In [ ]:
class HandleCategoricals(BaseEstimator, TransformerMixin):
    def fit(self, X, y = None):     
        self.object_cols = list(X.dtypes[X.dtypes == object].index)
        print("found these categoricals", self.object_cols)
        self.encoders = {}
        for col in self.object_cols:
            self.encoders[col] = {c: i for i, c in enumerate(X[col].unique())}

        return self

    def transform(self, X, y = None):
        X2 = X.copy()
        for col in self.object_cols:
            X2[col] = X2[col].map(self.encoders[col]).fillna(-1).astype(int)
        return X2    

In [ ]:
myHandleCategoricals = HandleCategoricals()
myPipeline = Pipeline(steps=[
    ('HandleDates', HandleDates()),
    ('HandleCategoricals', myHandleCategoricals)
])
myPipeline.fit(X)
X_transformed = myPipeline.transform(X)

In [ ]:
# class Pipeline:
#     object_cols = None
    
#     def transform(self, X):

#         X2 = X.copy()
#         date_series = pd.to_datetime(X['time'])

#         date_accessor = date_series.dt
#         for attr in ['date', 
#                      'day', 'day_of_week', 'day_of_year', 
#                      'days_in_month', 'freq', 'hour', 
#                      'is_leap_year', 'is_month_end', 'is_month_start', 'is_quarter_end', 
#                      'is_quarter_start', 'is_year_end', 'is_year_start', 
#                      'microsecond', 'minute', 'month', 'nanosecond', 
#                       'quarter', 'second', 'time', 
#                      'weekday', 'year'
#                     ]:
#             X2[attr] = getattr(date_accessor, attr)
#         X2['week'] = date_accessor.isocalendar().week.astype(int)
#         X2['original_time'] = X['time']
#         X2['time'] = X2['time'].astype(str)
        
#         # add location as one column
#         # this makes the eval better and kaggle worse. Commenting out for now.
# #         X2['location'] = X2[["x","y"]].apply(tuple, axis=1)
        
#         # turn other objects into strings.
#         self.object_cols = list(X2.dtypes[X2.dtypes == object].index)
#         X2[self.object_cols] = X2[self.object_cols].astype(str)
        
        
#         return X2

In [ ]:
# myPipeline = Pipeline()
# X_transformed = myPipeline.transform(X)

In [ ]:
%%time
# models by location
models_by_location = {}
scores = []; weights = []
for (x_loc, y_loc) in locations:
    print("location", x_loc, y_loc)
    subset_index = X_transformed.query(f"x == {x_loc} and y == {y_loc}").index
    models = []
    for i, (X_train, X_eval, y_train, y_eval) in enumerate(
        train_eval_kfolds(X_transformed.loc[subset_index], y.loc[subset_index], random_state=20220301)):

        print("split", i)
        model = CatBoostRegressor(verbose=0, cat_features=myHandleCategoricals.object_cols, 
                              random_state=20220301, iterations=iterations) 
#         model = LGBMRegressor(random_state=20220305)
        model.fit(X_train, y_train, eval_set=(X_eval, y_eval))
        score = mean_absolute_error(model.predict(X_eval), y_eval)
        print('\tmae', score)
        scores.append(score)
        weights.append(len(y_eval)) # to compute weighted average
        models.append(model)
    models_by_location[(x_loc, y_loc)] = models

In [ ]:
score_models_by_location = np.average(scores, weights=weights)
print("average mean of models by location", score_models_by_location)

In [ ]:
%%time
scores = []
overall_models = []
for i, (X_train, X_eval, y_train, y_eval) in enumerate(
    train_eval_kfolds(X_transformed, y, random_state=20220301)):
    
    print("split", i)
    model = CatBoostRegressor(verbose=100, cat_features=myHandleCategoricals.object_cols, 
                          random_state=20220301, iterations=iterations)   
#     model = LGBMRegressor(random_state=20220305)
    model.fit(X_train, y_train, eval_set=(X_eval, y_eval))
    score = mean_absolute_error(model.predict(X_eval), y_eval)
    print('\tmae', score)
    scores.append(score)
    overall_models.append(model)
    

In [ ]:
score_overall_model = np.mean(scores)
print("mae for overall model", score_overall_model)

In [ ]:
print("overall eval score", np.mean([score_overall_model, score_models_by_location]))

In [ ]:
test_df = pd.read_csv(
    '../input/tabular-playground-series-mar-2022/test.csv', index_col='row_id')
test_df.head()

In [ ]:
test_df_transformed = myPipeline.transform(test_df)
test_df_transformed

In [ ]:
submission_df = test_df[[]].copy()

In [ ]:
for (x_loc, y_loc) in locations:
    subset_index = test_df_transformed.query(f"x == {x_loc} and y == {y_loc}").index
    models = models_by_location[(x_loc, y_loc)]
    all_test_preds = []
    for model in models:
        test_preds = model.predict(test_df_transformed.loc[subset_index])
        all_test_preds.append(test_preds)
    submission_df.loc[subset_index, "by_location_pred"] = sum(all_test_preds) / len(all_test_preds)  

In [ ]:
submission_df

In [ ]:
all_test_preds = []
for model in overall_models:
    test_preds = model.predict(test_df_transformed)
    all_test_preds.append(test_preds)

In [ ]:
ensembled_test_preds = sum(all_test_preds) / len(all_test_preds)

In [ ]:
submission_df['overall_model'] = ensembled_test_preds

In [ ]:
submission_df

In [ ]:
submission_df['congestion'] = submission_df[['overall_model', 'by_location_pred']].mean(
    axis=1)

In [ ]:
# round - see if that helps
submission_df['congestion'] = submission_df['congestion'].round().astype(int)

In [ ]:
submission_df[['congestion']].to_csv(f'submission_run{run}.csv')

In [ ]:
submission_df